In [1]:
!pip install pytelegrambotapi -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.2/232.2 kB 4.9 MB/s eta 0:00:00


In [2]:
!pip install -U sentence-transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.5/149.5 kB 3.4 MB/s eta 0:00:00


In [3]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer

from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import random

In [4]:
import telebot

In [5]:
base_model = 'sentence-transformers/msmarco-bert-base-dot-v5'
tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModel.from_pretrained("/content/model")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [6]:
df = pd.read_csv('/content/Data/Barney.csv')

In [7]:
def get_embeddings_from_dataset(dataset, tokenizer, model):
    embeddings = []
    for q in dataset['Q']:
        encoded_q = tokenizer(q, return_tensors='pt')
        with torch.no_grad():
            q_embedding = model(**encoded_q).pooler_output
        embeddings.append(q_embedding)
    return embeddings

In [8]:
# Getting embeddings from the dataset
embeddings = get_embeddings_from_dataset(df, tokenizer, model)

In [9]:
def find_similar_answers_with_noise(question, dataset, tokenizer, model, embeddings, top_n=5):
    similarities = []
    encoded_input = tokenizer(question, return_tensors='pt')
    with torch.no_grad():
        question_embedding = model(**encoded_input).pooler_output

    for q_embedding in embeddings:
        similarity = cosine_similarity(question_embedding, q_embedding)
        similarity_with_noise = similarity.item() + random.uniform(-0.1, 0.1)
        similarities.append(similarity_with_noise)

    top_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:top_n]

    similar_answers = [(dataset['A'][idx], similarities[idx]) for idx in top_indices]
    return similar_answers

In [10]:
token = 'your token'

In [11]:
# Создаем экземпляр бота
bot = telebot.TeleBot(token)

In [12]:
# Функция, обрабатывающая команду /start
@bot.message_handler(commands=["start"])
def start(m):
    bot.send_message(m.chat.id, "Hi, bro! I'm Barbey)")

# Получение сообщений от юзера
@bot.message_handler(content_types=["text"])
def handle_text(message):
    try:
        # Получаем ответы от модели
        similar_answers = find_similar_answers_with_noise(message.text, df, tokenizer, model, embeddings)

        # Отправляем один из ответов пользователю
        bot.send_message(message.chat.id, similar_answers[0][0])
    except Exception as e:
        print("Exception:", e)
        bot.send_message(message.chat.id, "Sorry, couldn't handle your request.")

In [13]:
# Запускаем бота
bot.polling(none_stop=True, interval=0)